In [2]:
import json
import os

In [3]:
os.getcwd()

'/Users/alexrotondo/Documents/nlp'

In [8]:
with open('../../Downloads/JEOPARDY_QUESTIONS1.json', 'r') as file:
    questions = json.load(file)

In [10]:
questions[0]

{'category': 'HISTORY',
 'air_date': '2004-12-31',
 'question': "'For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory'",
 'value': '$200',
 'answer': 'Copernicus',
 'round': 'Jeopardy!',
 'show_number': '4680'}

In [29]:
def preprocess(questions):
    processedQuestions = []
    for question in questions:
        processedQuestions += ['<bos> ' + question['question'].lower()[1:-1] + ' <eos>']
    return processedQuestions

In [30]:
processedQuestions = preprocess(questions)

In [31]:
n = len(processedQuestions)

In [32]:
n

216930

In [33]:
processedQuestions[2]

'<bos> the city of yuma in this state has a record average of 4,055 hours of sunshine each year <eos>'

In [34]:
def get_n_grams(n, questions):
    ngrams = {}
    for question in questions:
        tokens = question.split(' ')
        grams = [tuple(tokens[i:i+n]) for i in range(len(tokens) - n)]
        for gram in grams:
            if gram in ngrams:
                ngrams[gram] += 1
            else:
                ngrams[gram] = 1
    return ngrams

In [35]:
bigrams = get_n_grams(2, processedQuestions)

In [36]:
def get_n_gram_probabilities(ngrams):
    n = sum(ngrams.values())
    for ngram in ngrams:
        ngrams[ngram] /= n
    return ngrams

In [37]:
bigramProbabilities = get_n_gram_probabilities(bigrams)